In [11]:
# Import necessary libraries
import ipywidgets as widgets
import pandas as pd
import joblib
from geopy.geocoders import Nominatim

# Load your pre-trained random forest model
model = joblib.load('randomforest.joblib')

# Create input widgets
num_rooms = widgets.FloatSlider(description='Rooms', min=1, max=10, step=0.5, value=3)
floor = widgets.IntSlider(description='Floor', min=0, max=20, value=5)
living_space = widgets.IntSlider(description='Living Space (sqft)', min=10, max=500, value=100)
address_input = widgets.Text(description='Address')
postal_code_input = widgets.Text(description='Postal Code')

# Create OK button
ok_button = widgets.Button(description='OK')

# Function to geocode address
def geocode_address(address):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address)
    return location.latitude, location.longitude

def predict_price(button):
    # Get user inputs
    latitude, longitude = geocode_address(address_input.value)
    if latitude is None or longitude is None:
        print("Unable to geocode address.")
        return
    features = pd.DataFrame({
        'Rooms': [num_rooms.value],
        'Floor': [floor.value],
        'LivingSpace': [living_space.value],
        'Latitude': [latitude],
        'Longitude': [longitude],
        'postalcode': postal_code_input.value,
    })
    
    # Predict price
    predicted_price = model.predict(features.values)[0]  # Use .values here

    # Display predicted price
    with price_output:
        print(f'Predicted Price: CHF {predicted_price:.2f}')

# Attach event handler to OK button
ok_button.on_click(predict_price)

# Create output widget for displaying predicted price
price_output = widgets.Output()

# Display widgets
widgets.VBox([num_rooms, floor, living_space, address_input, postal_code_input, ok_button, price_output])